# Preprocessing: SLCH002


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [25]:
#%matplotlib qt5

In [2]:
## Prep paths ##

subject = 'SLCH002'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [66]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2373999 =      0.000 ...  1186.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_notched_filtered_clean_ieeg-1.fif...
    Range : 2374000 ... 2846599 =   1187.000 ...  1423.300 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_raw_clean_ieeg.fif...
    Range : 0 ... 2373999 =      0.000 ...  1186.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/SLCH002/ieeg/SLCH002_raw_clean_ieeg-1.fif...
    Range : 2374000 ... 2846599 =   1187.000 ...  1423.300 secs
Ready.


## Bipolar Rereferencing

In [67]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [68]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

REF1 noref
REF2 noref
A1 A2
A2 A3
A3 A4
A4 A5
A5 A6
A6 A7
A7 A8
A8 A9
A9 A10
A10 A11
A11 A12
A12 A13
A13 A14
A14 A15
A15 A16
B1 B2
B2 B3
B3 B4
B4 B5
B5 B6
B6 B7
B7 B8
B8 B9
B9 B10
B10 B11
B11 B12
B12 B13
B13 B14
B14 B15
B15 B16
C1 C2
C2 C3
C3 C4
C4 C5
C5 C6
C6 C7
C7 C8
C8 C9
C9 C10
C10 C11
C11 C12
C12 C13
C13 C14
C14 C15
C15 C16
D1 D2
D2 D3
D3 D4
D4 D7
D5 noref
D6 noref
D7 D8
D8 D9
D9 D10
E1 E2
E2 E3
E3 E4
E4 E5
E5 E6
E6 E8
E7 noref
E8 E9
E9 E10
F1 F2
F2 F3
F3 F4
F4 F5
F5 F6
F6 F7
F7 F8
F8 F9
F9 F10
G1 G2
G2 G3
G3 G4
G4 G5
G5 G6
G6 G7
G7 G8
G8 G9
G9 G10
h1 h2
h2 h3
h3 h4
h4 h5
h5 h6
h6 h7
h7 h8
h8 h9
h9 h10
i1 i2
i2 i3
i3 i6
i4 noref
i5 noref
i6 i7
i7 i8
i8 i9
i9 i10
i10 i11
i11 i12
i12 i13
i13 i14
EKG1 noref
EKG2 noref
J1 noref
J2 noref
J3 noref
J4 J5
J5 J6
J6 J7
J7 J8
J8 J9
J9 J10
J10 J11
J11 J12
J12 J13
J13 J14
K1 K2
K2 K3
K3 K4
K4 K5
K5 K6
K6 K7
K7 K8
K8 K9
K9 K10
K10 K11
K11 K12
K12 K13
K13 K14
K14 K15
K15 K16
L1 L2
L2 L3
L3 L4
L4 L5
L5 L6
L6 L7
L7 L8
L8 L9
L9 L10
L10 L11
L11 L12


In [56]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

In [ ]:
# Visualize it #

filtered_notch_bp_data.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## White Matter Rereferencing

In [85]:
hippocampal_contacts = [x for x in anode if x.startswith("J") or x.startswith("K")]
cathode_hc =  ["J9" if not hippocampal_contacts[i].startswith("K") else "K8" for i, val in enumerate(hippocampal_contacts)]
anode_hc = hippocampal_contacts.copy()
hippocampal_contacts.extend(["STI"])

In [87]:
raw_clean_fif.pick_channels(hippocampal_contacts)

<Raw | SLCH002_raw_clean_ieeg.fif, 26 x 2846600 (1423.3 s), ~32 kB, data not loaded>

In [88]:
raw_clean_fif.info['ch_names']


['J4',
 'J5',
 'J6',
 'J7',
 'J8',
 'J9',
 'J10',
 'J11',
 'J12',
 'J13',
 'K1',
 'K2',
 'K3',
 'K4',
 'K5',
 'K6',
 'K7',
 'K8',
 'K9',
 'K10',
 'K11',
 'K12',
 'K13',
 'K14',
 'K15',
 'STI']

In [91]:
## Apply Rereference #

# load raw data
raw_clean_fif.load_data()

# set raw reference 
bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode_hc, cathode = cathode_hc)

# save
bp_raw_fif.save(f"{preproc_data_dir}/{subject}_wm_raw_clean_data.fif", overwrite = True)


# clear var
bp_raw_fif = []
raw_clean_fif = []    
    
    

sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=25, n_times=2846600
    Range : 0 ... 2846599 =      0.000 ...  1423.300 secs
Ready.
Added the following bipolar channels:
J4-J9, J5-J9, J6-J9, J7-J9, J8-J9, J9-J9, J10-J9, J11-J9, J12-J9, J13-J9, K1-K8, K2-K8, K3-K8, K4-K8, K5-K8, K6-K8, K7-K8, K8-K8, K9-K8, K10-K8, K11-K8, K12-K8, K13-K8, K14-K8, K15-K8
Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif


/tmp/ipykernel_82199/923386857.py:10: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_wm_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif
[done]


## Epoching the data 

Longest usable trial is 8.45 seconds, round up to 9 and add 3 seconds for buffer. 1 second baseline if we want, plus 3 seconds buffer

### Onset

In [3]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif...


/tmp/ipykernel_233022/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.
480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...


/tmp/ipykernel_233022/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...


### Trial End

In [5]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1]
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.


/tmp/ipykernel_233022/1129154783.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts


/tmp/ipykernel_233022/1129154783.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...
Loading data for 119 events and 20001 original time points ...


### Last Away

In [6]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,8,39400,0,1
1,1,12,59400,0,1
2,2,3,71000,0,1
3,4,16,110400,0,1
4,5,2,126000,0,1
...,...,...,...,...,...
218,234,10,2740200,0,1
219,235,19,2750000,0,1
220,236,2,2761200,0,1
221,237,18,2772600,0,1


In [7]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [8]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.
Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 112 events and 20001 original time points ...


/tmp/ipykernel_233022/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_233022/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 111 events and 20001 original time points ...


### First Dot

In [9]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,8,37400,0,1
1,1,12,54200,0,1
2,2,3,69200,0,1
3,3,7,87600,0,1
4,4,16,108200,0,1
...,...,...,...,...,...
219,234,10,2738800,0,1
220,235,19,2747600,0,1
221,236,2,2759600,0,1
222,237,18,2770000,0,1


In [10]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [11]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 112 events and 18001 original time points ...


/tmp/ipykernel_233022/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_233022/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 112 events and 18001 original time points ...


## First Move

In [12]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,8,36600,0,1
1,1,12,53800,0,1
2,2,3,69000,0,1
3,3,7,87400,0,1
4,4,16,108000,0,1
...,...,...,...,...,...
234,234,10,2738800,0,1
235,235,19,2747400,0,1
236,236,2,2758800,0,1
237,237,18,2769200,0,1


In [13]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [14]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_233022/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_233022/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 119 events and 20001 original time points ...


### White Matter Reference Presentation

In [16]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_wm_raw_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_wm_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif...
    Range : 0 ... 2846599 =      0.000 ...  1423.300 secs
Ready.


/tmp/ipykernel_233022/2012182281.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_wm_raw_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 32001 original time points ...
Loading data for 240 events and 32001 original time points ...


/tmp/ipykernel_233022/2012182281.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_wm_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### WM Reference Last Away

In [ ]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_wm_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_wm_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif...
    Range : 0 ... 2846599 =      0.000 ...  1423.300 secs
Ready.


/tmp/ipykernel_3967/3216611557.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_wm_raw_clean_data.fif")


Not setting metadata
223 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 223 events and 20001 original time points ...


/tmp/ipykernel_3967/3216611557.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_wm_raw_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_wm_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


## Ghost Attack

In [3]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,3,7,89200,0,1
1,6,10,142400,0,1
2,7,11,160200,0,1
3,14,6,247800,0,1
4,19,14,314600,0,1
5,22,8,359000,0,1
6,34,2,497400,0,1
7,35,6,509000,0,1
8,39,16,554200,0,1
9,41,2,581400,0,1


In [4]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [5]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif...
    Range : 0 ... 2529999 =      0.000 ...  1264.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data-1.fif...
    Range : 2530000 ... 2846599 =   1265.000 ...  1423.300 secs
Ready.
Not setting metadata
30 matching events found


/tmp/ipykernel_66159/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 30 events and 20001 original time points ...


/tmp/ipykernel_66159/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/SLCH002/ieeg/SLCH002_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
